## Tweet classification

File names

In [1]:
base_location = './'

# To read
tweets_raw_file = base_location + 'footballsentiment_task.csv'
tweets_run_file = base_location + 'footballsentiment_task_run.csv'
corpus_tweets_2012_xml = base_location + 'general-train-tagged-3l.xml'
corpus_tweets_2017_xml = base_location + 'intertass-train-tagged.xml'

# To generate
corpus_tweets_2012_csv = base_location + 'general-train-tagged-3l.csv'
corpus_tweets_2017_csv = base_location + 'intertass-train-tagged.csv'
corpus_tweets_csv = base_location + 'corpus_tweets.csv'

Import libraries

In [2]:
import pandas as pd
import numpy as np

### Load datasets

In [3]:
tweets_raw = pd.read_csv(tweets_raw_file, encoding='utf-8')
tweets_run = pd.read_csv(tweets_run_file, encoding='utf-8')

print('Total tweets: %d' % len(tweets_raw))
print('Evaluated tweets so far: %d' % len(tweets_run))

Total tweets: 411
Evaluated tweets so far: 177


### Create train and test data

Aux function to convert evaluations to numeric values, accordnig to the rule:
- `Negativo`: -1.
- `Positivo`: 1.
- `Neutro`: 0.
- Anything else: 2.

In [4]:
def convert_to_numeric(evaluation):
    if evaluation == 'Positivo':
        return 1
    elif evaluation == 'Neutro':
        return 0
    elif evaluation == 'Negativo':
        return -1
    else:
        return 2    

Build new array of dictionaries with keys `id` (the task ID), `tweet` (the tweet string) and `score` (the tweet evaluation) by joining data from both CSV files.

In [10]:
# Build dictionary of tweets where key is the task__id
tweets_obj = {}
for index, row in tweets_raw.iterrows():
    tweets_obj[row.task__id] = row.taskinfo__Tweet

# Build dictinary of tweet scores where key is the task_id
scores_obj = {}
for index, row in tweets_run.iterrows():
    scores_obj[row.task_run__task_id] = row.task_run__info

# Create final wteets dictionary
own_tweets = []
for i, key in enumerate(scores_obj):
    own_tweets.append({
        'id': key,
        'tweet': tweets_obj[key], 
        'score': convert_to_numeric(scores_obj[key])
    })

print('Total different tweets evalauted so far: %d' % len(own_tweets))

Total different tweets evalauted so far: 588


### POS Tagging

Import libraries to read XML:

In [11]:
from lxml import objectify

Import/read most recent corpus (2017):

In [13]:
# 4 values of sentiment: N, P, NONE, NEU
xml = objectify.parse(open(corpus_tweets_2017_xml))
root = xml.getroot()
general_tweets_corpus_train_2017 = pd.DataFrame(columns=('content', 'polarity'))
tweets = root.getchildren()
for i in range(0, len(tweets)):
    tweet = tweets[i]
    row = dict(zip(['content', 'polarity'], [tweet.content.text, tweet.sentiment.polarity.value.text]))
    row_s = pd.Series(row)
    row_s.name = i
    general_tweets_corpus_train_2017 = general_tweets_corpus_train_2017.append(row_s)
    
general_tweets_corpus_train_2017.to_csv(corpus_tweets_2017_csv, index=False, encoding='utf-8')

Import/read biggest corpus (2012), to concatenate it with the previous noe:

In [14]:
# 4 values of sentiment: N, P, NONE, NEU
xml = objectify.parse(open(corpus_tweets_2012_xml))
root = xml.getroot()
general_tweets_corpus_train_2012 = pd.DataFrame(columns=('content', 'polarity'))
tweets = root.getchildren()
for i in range(0, len(tweets)):
    tweet = tweets[i]
    row = dict(zip(['content', 'polarity'], [tweet.content.text, tweet.sentiments.polarity.value.text]))
    row_s = pd.Series(row)
    row_s.name = i
    general_tweets_corpus_train_2012 = general_tweets_corpus_train_2012.append(row_s)
    
general_tweets_corpus_train_2012.to_csv(corpus_tweets_2012_csv, index=False, encoding='utf-8')

Concatenate general corpus dataset with 2017 one, to have a better result:

In [45]:
tweets_corpus = pd.concat([
        general_tweets_corpus_train_2012,
        general_tweets_corpus_train_2017
    ])
tweets_corpus.sample(10)

,content,polarity
472,"Cruz está que de sale RT ""@agusperezblanco: Mi...",P
6175,“@VaquerBoy: @David_Busta Recuerda que no eres...,P
4932,"Tres años de instrucción, uno y medio de suspe...",N
3095,Preguntan a Valcárcel si aún quiere devolver c...,NEU
352,Cameron rechaza reformar el Tratado de Lisboa ...,NEU
5670,"Opel ha sido, es y será empresa estratégica pa...",P
613,"¿""Agresión con cámara fotográfica""? Cuesta muc...",N
6583,El Supremo juzga a Otegui. repasamos los que h...,NEU
6279,Telefónica contrata a marido d la Vicep. del G...,N
2435,Ya está aqui Bern Shcuster!!!;)) http://t.co/5...,P


In [48]:
print('Total corpus tweets: %d' % len(tweets_corpus))

Total corpus tweets: 6605


Remove tweets without polarity (polarity `NONE`):

In [47]:
tweets_corpus = tweets_corpus.query('polarity != "NONE"')

Remove links:

In [28]:
tweets_corpus = tweets_corpus[-tweets_corpus.content.str.contains('^http.*$')]

In [53]:
import re
url_regex = re.compile('https?:\/\/t\.co\/[\w]{8,8}')
tweets_corpus_no_links = tweets_corpus
tweets_corpus_no_links['content'] = tweets_corpus_no_links['content'].map(lambda x: re.sub(url_regex, '', x))

In [55]:
print('Total corpus tweets after cleaning: %d' % len(tweets_corpus_no_links))

Total corpus tweets after cleaning: 6605


In [56]:
tweets_corpus.sample(10)

,content,polarity
1109,"@PabloAIglesias no se si mañana o pasado,jaja",P
2783,"#Andalucia necesita claridad en sus cuentas, t...",N
2263,El Gobierno recuerda a sindicatos y empresario...,N
6233,Según el relato de EP la única presión que se ...,N
3562,Si Chacón gana es urgente que aprenda la difer...,NEU
1284,Es lo maravilloso de este nuevo medio de expre...,P
330,Resumen de la larga y agria noche en Bruselas:...,N
4658,Bruselas será ligeramente flexible con los obj...,N
6951,Desde que en julio de 1988 empecé a trabajar e...,N
1240,Hoy en Cartaya con las personas mayores. Nos m...,P


### Tokenization and stemming

In [57]:
#download spanish stopwords
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/david.santosg/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [58]:
from string import punctuation
non_words = list(punctuation)

#we add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
non_words

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 '¿',
 '¡',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [59]:
from sklearn.feature_extraction.text import CountVectorizer       
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = SnowballStemmer('spanish')
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # remove non letters
    text = ''.join([c for c in text if c not in non_words])
    # tokenize
    tokens =  word_tokenize(text)

    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

### Model Evaluation

In [60]:
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

/opt/Conda.io/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [61]:
tweets_corpus_no_links['polarity_bin'] = 0
tweets_corpus_no_links.polarity_bin[tweets_corpus_no_links.polarity.isin(['P'])] = 1
tweets_corpus_no_links.polarity_bin[tweets_corpus_no_links.polarity.isin(['N'])] = -1
tweets_corpus_no_links.polarity_bin.value_counts(normalize=True)

/opt/Conda.io/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/Conda.io/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


 1    0.484784
-1    0.393641
 0    0.121575
Name: polarity_bin, dtype: float64

In [64]:
from sklearn.model_selection import GridSearchCV
vectorizer = CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords)

pipeline = Pipeline([
    ('vect', vectorizer),
    ('cls', LinearSVC()),
])



parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'cls__C': (0.2, 0.5, 0.7),
    'cls__loss': ('hinge', 'squared_hinge'),
    'cls__max_iter': (500, 1000)
}


grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1 , scoring='roc_auc')
grid_search.fit(tweets_corpus_no_links.content, tweets_corpus_no_links.polarity_bin)

JoblibLookupError: JoblibLookupError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/Conda.io/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.6/site-packages/ipykernel/__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/opt/Conda.io/lib/python3.6/runpy.py

In [65]:
grid_search.best_params_

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [66]:
from sklearn.externals import joblib
joblib.dump(grid_search, 'grid_search.pkl')

['grid_search.pkl']

In [67]:


model = LinearSVC(C=.2, loss='squared_hinge',max_iter=1000,multi_class='ovr',
              random_state=None,
              penalty='l2',
              tol=0.0001
)

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = spanish_stopwords,
    min_df = 50,
    max_df = 1.9,
    ngram_range=(1, 1),
    max_features=1000
)

corpus_data_features = vectorizer.fit_transform(tweets_corpus.content)
corpus_data_features_nd = corpus_data_features.toarray()



LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/david.santosg/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [68]:
scores = cross_val_score(
    model,
    corpus_data_features_nd[0:len(tweets_corpus)],
    y=tweets_corpus.polarity_bin,
    scoring='roc_auc',
    cv=5
    )

scores.mean()

NameError: name 'corpus_data_features_nd' is not defined

### Polarity Prediction

In [69]:
tweets = pd.read_csv('tweets_parsed.csv', encoding='utf-8')

FileNotFoundError: File b'tweets_parsed.csv' does not exist

In [70]:
tweets = tweets[tweets.tweet.str.len() < 150]
tweets.lat = pd.to_numeric(tweets.lat, errors='coerce')
tweets = tweets[tweets.lat.notnull()]


#We make sure only those tweets in the Murcia bounding box are kept
min_lon = -1.157420
max_lon = -1.081202
min_lat = 37.951741
max_lat = 38.029126

tweets = tweets[(tweets.lat.notnull()) & (tweets.lon.notnull())]

tweets = tweets[(tweets.lon > min_lon) & (tweets.lon < max_lon) & (tweets.lat > min_lat) & (tweets.lat < max_lat)]
tweets.shape

AttributeError: 'list' object has no attribute 'tweet'